# Youtube Analysis 
Proof of concept project for analysing the sentiment of videos returned by searching for a key word (i.e. product name). 

Initial implimentation uses NLP to assess the polarity of comments for each video combining this with other metrics to return an overall sentiment score.

Future additions will integrate a company product list and security identifiers to link companies/ tickers to sentiment scores, allowing for statistical analysis of 


## Setup

1. Create Youtube Data API key as per the instructions [here](https://developers.google.com/youtube/v3/getting-started) 

2. Create `.env` file containing the key as follows: `YT_API_KEY=[key]`
   
3. Finally, after installing SpaCy in your environment, ensure the language library is installed by running the below command in the terminal:

```shell
                                        spacy download en_core_web_sm
```

#### Dependencies

In [1]:
import os
import pandas as pd

import database
from data_api import youtube
from helpers import dict_search, min_max_scaler

##### Import Google Data API Key and Initiate Youtube API Class Instance


In [2]:
# Import env variables and set API key
from dotenv import load_dotenv
load_dotenv()

DEVELOPER_KEY = os.environ.get('YT_API_KEY')


# Create YouTube Data API object
yt = youtube(DEVELOPER_KEY)

## Search
Run a search using the key term, returning the IDs of relevant videos ordered by upload date.

In [3]:
keyword = 'macbook' # Using macbooks as an example

# Use search method to retrieve IDs
response = yt.search(keyword, order='date')
raw_ids = dict_search(response, ["videoId"], list_depth=2)
ids = [row['videoId'] for row in raw_ids]

In [4]:
# Retrieve general information for each video
raw_stats = yt.video_stats(ids)
clean_stats = dict_search(raw_stats, [
    "id", 
    "title",
    "decription", 
    "channelTitle",
    "channelId", 
    "categoryId", 
    "viewCount", 
    "likeCount", 
    "commentCount", 
    "publishedAt"], list_depth=2)
stats_df = pd.DataFrame(clean_stats)
stats_df.head(5)

,id,publishedAt,channelId,title,channelTitle,categoryId,viewCount,likeCount,commentCount
0,GCh9jW6HLPk,2022-01-31T18:00:08Z,UCrHLsHcDSEgtIc-aY-fUhTg,"ATENCIÓN⚠️ MacBook Air M2, MacBook Pro 13, Mac...",Cristo Vega,28,44,10,2
1,PuuiGG9-8EE,2022-01-31T18:00:09Z,UCqVWosEnWaUa4ADdUKl3Kew,What's On My MacBook Pro,AlexanderSucksAtYouTube,20,0,1,2
2,5WChMSBtOH0,2022-01-31T16:48:43Z,UCxO46fGxONZ6CbRs58MtNSg,Il DeskSetup per QUALSIASI cosa! PS5 - Macbook...,BasemenTech,28,515,97,14
3,j8UzFJY2Sl4,2022-01-31T16:01:40Z,UCNqT8Qk747G4alIBzPgRQiw,بررسی جدیدترین شاهکار اپل مک بوک پرو ام ۱ پرو ...,Binosha - بینوشا,28,549,91,34
4,o7Ld925BGDI,2022-01-31T16:00:21Z,UCFO8al46IxCIOewbTo_zcRg,"Какие iPhone, Macbook, iPad, Apple Watch, AirP...","Эй, привет!",28,58,12,2


In [5]:
# Retrieve top level comment threads for each video to be used to guage polarity
raw_comments = yt.commentThread(ids)
comments = dict_search(raw_comments, [
    "videoId",
    "textDisplay",
    "publishedAt"
    ], list_depth=2)
comments_df = pd.DataFrame(comments)

In [6]:
# Rename id, comments and comment publishedAt columns and merge with stats dataframe
stats_df.rename(columns={'id':'videoId'}, inplace=True)
comments_df.rename(columns={'textDisplay':'comment', 'publishedAt':'commentDate'}, inplace=True)
merged_df = pd.merge(stats_df, comments_df, how='left', on='videoId')
merged_df['comment'].fillna('', inplace=True)

In [7]:
# Retrieve channel stats for each video and merge with other dataframe
raw_channelStats = yt.channel(stats_df['channelId'].to_list(), part="statistics")
channelStats = dict_search(raw_channelStats, [
    "id", 
    "subscriberCount", 
    "videoCount"
    ], list_depth=2)
channel_df = pd.DataFrame(channelStats)

# Rename ID column and merge
channel_df.rename(columns={'id':'channelId'}, inplace=True)
merged_df = pd.merge(merged_df, channel_df, how='left', on='channelId')

## Sentiment Analysis

We assess that sentiment can be summarised by: <br>
<br>
$\text{Sentiment} = \dfrac{\sum\text{Comment Polarity}}{\text{Video comment Count}} \times \dfrac{\text{Video Views}}{\text{Channel Subscribers}}$

In [8]:
# Import sentiment object for analysis
from analysis import sentiment

# Analyse each comment and give polarity score
# 1: Positive, 0: Neutral, -1: Negative
comment_list = merged_df['comment'].astype(str).to_list()
s = sentiment(comment_list)
merged_df['comment_polarity'] = s.polarity()

In [9]:
# amend data type in count columns from string to integers
merged_df[['likeCount', 'viewCount', 'commentCount', 'subscriberCount']] = merged_df[['likeCount', 'viewCount', 'commentCount', 'subscriberCount']].astype(int)
merged_df[['publishedAt', 'commentDate']] = merged_df[['publishedAt', 'commentDate']].astype('datetime64').dt.normalize()

df = merged_df.copy()

# Polarity scaled by comment count
df['comment_polarity'] /= df['commentCount']

df['view_sub_ratio'] = df['viewCount'] / df['subscriberCount']
df['like_view_ratio'] = df['likeCount'] / df['viewCount']
df['comment_view_ratio'] = df['commentCount'] / df['viewCount']

AttributeError: 'DataFrame' object has no attribute 'dt'

In [ ]:
import matplotlib.pyplot as plt

# Sentiment time series
time_series = df.copy()
time_series = time_series.groupby('commentDate').agg(polarity=('comment_polarity','mean'), count = ('commentDate','size')).reset_index()
time_series.fillna(0, inplace=True)

TypeError: complex() first argument must be a string or a number, not 'datetime.date'

In [13]:
# Normalise comment polarity
time_series['comment_polarity'] = min_max_scaler(time_series['comment_polarity'])


In [ ]:

# Groupby, summing polarity of comments for each video ID
df = df.groupby(['videoId','view_sub_ratio', 'like_view_ratio', 'comment_view_ratio', 'subscriberCount']).agg({'comment_polarity':['sum']}).reset_index()
df.columns = df.columns.droplevel(1)

# Create video sentiment score
df['sentiment'] = df['comment_polarity']*df['view_sub_ratio']

df.head(5)


## Stock Price Predictive Power

\*\*TODO\*\*